In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms, models
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from collections import Counter

In [ ]:
train_dir = "data/train"
val_dir = "data/test"

In [ ]:
# ✅ Enhanced transforms with augmentation
train_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ✅ Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

In [ ]:
print("Class distribution:")

class_counts = Counter([train_dataset.targets[i] for i in range(len(train_dataset))])
for i, class_name in enumerate(train_dataset.classes):
    print(f"{class_name}: {class_counts[i]} samples")

In [ ]:
# ✅ Calculate class weights for balanced loss
class_sample_counts = [class_counts[i] for i in range(len(train_dataset.classes))]
weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
sample_weights = weights[train_dataset.targets]

# Create weighted sampler for balanced batches
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

In [ ]:
# ✅ Data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=32, 
    sampler=sampler,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=32, 
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True

num_classes = len(train_dataset.classes)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)
model = model.to(device)

# ✅ Weighted loss function to handle class imbalance
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_dataset.targets),
    y=train_dataset.targets
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)

# ✅ Optimizer with different learning rates for different parts
optimizer = optim.AdamW([
    {'params': model.layer4.parameters(), 'lr': 1e-4},  # Lower LR for pretrained layers
    {'params': model.fc.parameters(), 'lr': 1e-3}       # Higher LR for new classifier
], weight_decay=1e-4)

# ✅ Better learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max', 
    factor=0.5, 
    patience=3
)

In [ ]:
num_epochs = 15
best_val_acc = -1.0  # So the first epoch always saves the model
patience_counter = 0
patience = 7

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 50)

    # -------- TRAINING --------
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # -------- VALIDATION --------
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    class_correct = [0.0 for _ in range(num_classes)]
    class_total = [0.0 for _ in range(num_classes)]

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            c = (predicted == labels).squeeze()
            for i in range(labels.size(0)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    val_loss = val_loss / len(val_loader)
    val_acc = val_correct / val_total

    # -------- LOGGING --------
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.4f}")
    print("Per-class accuracy:")
    for i in range(num_classes):
        if class_total[i] > 0:
            acc = class_correct[i] / class_total[i]
            print(f"  {train_dataset.classes[i]}: {acc:.4f}")

    # -------- SCHEDULER --------
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_acc)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr != old_lr:
        print(f"Learning rate reduced from {old_lr:.6f} to {new_lr:.6f}")

    # -------- EARLY STOPPING --------
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), 'models/model.pth')
        print(f"New best model saved with validation accuracy: {best_val_acc:.4f}")
    else:
        patience_counter += 1
        print(f"No improvement. Patience counter: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

print(f"\nTraining completed. Best validation accuracy: {best_val_acc:.4f}")


In [ ]:
model.load_state_dict(torch.load('models/model.pth'))
print("Best model loaded for final evaluation.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Get a batch of validation data
inputs, labels = next(iter(val_loader))
inputs = inputs.to(device)
labels = labels.to(device)

model.eval()
with torch.no_grad():
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)

# Move tensors to CPU for visualization
inputs = inputs.cpu()
labels = labels.cpu()
preds = preds.cpu()

# Unnormalize images for display
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.axis('off')

# Plot the first 6 images with predictions
plt.figure(figsize=(12, 6))
for i in range(6):
    plt.subplot(2, 3, i+1)
    imshow(inputs[i], title=f"Pred: {train_dataset.classes[preds[i]]}\nTrue: {train_dataset.classes[labels[i]]}")
plt.tight_layout()
plt.show()

In [ ]:
import cv2
import torch
from PIL import Image
from torchvision import transforms
import time

# Load model
model.load_state_dict(torch.load("models/model.pth", map_location=device))
model.to(device)
model.eval()

# Class names from training dataset
class_names = train_dataset.classes

# Transform used during training (update this to match your training preprocessing)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # adjust based on your model input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],   # ImageNet means/std if pretrained
                         [0.229, 0.224, 0.225])
])

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Error: Cannot access webcam.")
    exit()

print("🟢 Webcam started. Press 'q' to quit.")

prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert frame to PIL image and apply transform
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    input_tensor = transform(image).unsqueeze(0).to(device)

    # Prediction
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)
        mood = class_names[predicted.item()]

    # FPS Calculation
    curr_time = time.time()
    fps = 1.0 / (curr_time - prev_time)
    prev_time = curr_time

    # Overlay mood and FPS on the frame
    cv2.putText(frame, f"Mood: {mood}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (255, 255, 0), 2)

    # Display the frame
    cv2.imshow('Live Mood Detection', frame)

    # Quit with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()